In [9]:
%load_ext autoreload
%autoreload 2
import torch
import copy
import os

import torch
import tqdm
import torchvision

from typing import Literal

import abstract_gradient_training as agt
from abstract_gradient_training import AGTConfig
from abstract_gradient_training.bounded_models import IntervalBoundedModel

import uci_datasets  # python -m pip install git+https://github.com/treforevans/uci_datasets.git
torch.manual_seed(0)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
batchsize = 1000000
data = uci_datasets.Dataset("houseelectric")
print(data)
x_train, y_train, x_test, y_test = data.get_split(split=0)

# Normalise the features and labels
x_train_mu, x_train_std = x_train.mean(axis=0), x_train.std(axis=0)
x_train = (x_train - x_train_mu) / x_train_std
x_test = (x_test - x_train_mu) / x_train_std
y_train_min, y_train_range = y_train.min(axis=0), y_train.max(axis=0) - y_train.min(axis=0)
y_train = (y_train - y_train_min) / y_train_range
y_test = (y_test - y_train_min) / y_train_range

# Form datasets and dataloaders
train_data = torch.utils.data.TensorDataset(torch.from_numpy(x_train).float(), torch.from_numpy(y_train).float())
test_data = torch.utils.data.TensorDataset(torch.from_numpy(x_test).float(), torch.from_numpy(y_test).float())
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batchsize, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=1000, shuffle=False)




houseelectric dataset, N=2049280, d=11


In [11]:
# batchsize = 1000000
# # configure the training parameters
# nominal_config = agt.AGTConfig(
#     fragsize=20000,
#     learning_rate=0.005,
#     epsilon=0.01,
#     k_private=1,
#     n_epochs=1,
#     device="cuda:0",
#     loss="mse",
#     log_level="DEBUG",
#     optimizer="SGDM", # we'll use SGD with momentum
#     optimizer_kwargs={"momentum": 0.9, "nesterov": True},
# )

# set up the AGT configuration
nominal_config = AGTConfig(
    fragsize=2000,
    learning_rate=0.25,
    n_epochs=50,
    device="cuda:1",
    l2_reg=0.01,
    k_private=1,
    loss="mse",
    log_level="INFO",
    lr_decay=2.0,
    clip_gamma=1.0,
    lr_min=0.001,
    optimizer="SGDM", # we'll use SGD with momentum
    optimizer_kwargs={"momentum": 0.9, "nesterov": True},
)

In [12]:
# to use privacy-safe certificates, we need to run AGT for a range of k_private values

# we'll just pick a reasonable range of k_private values. adding more values will increase the runtime
# but also result in tighter privacy results. even a few values are sufficient to demonstrate tighter privacy

k_private_values = [1, 2, 5, 10, 20, 50, 100] 
privacy_bounded_models = {}
config = copy.deepcopy(nominal_config)
# config.log_level = "WARNING"

for k_private in tqdm.tqdm(k_private_values):
    # update config
    config.k_private = k_private
    # form bounded model
    torch.manual_seed(1)
    # get the nn model
    model = torch.nn.Sequential(torch.nn.Linear(11, 64), torch.nn.ReLU(), torch.nn.Linear(64, 1)).to(config.device)
    bounded_model = IntervalBoundedModel(model, trainable=True)
    # dl_train = torch.utils.data.DataLoader(dataset_train, batch_size=batchsize, shuffle=True)
    # run AGT
    agt.privacy_certified_training(bounded_model, config, train_loader, test_loader, k=k_private)
    privacy_bounded_models[k_private] = bounded_model

  0%|          | 0/7 [00:00<?, ?it/s]

[AGT] [INFO    ] [12:00:07] =================== Starting Privacy Certified Training ===================
[AGT] [INFO    ] [12:00:07] Starting epoch 1
[AGT] [INFO    ] [12:00:23] Batch 1. Loss (mse): 0.340 <= 0.340 <= 0.340
[AGT] [INFO    ] [12:02:33] Starting epoch 2
[AGT] [INFO    ] [12:02:47] Batch 2. Loss (mse): 2.222 <= 2.222 <= 2.222
[AGT] [INFO    ] [12:05:01] Starting epoch 3
[AGT] [INFO    ] [12:05:14] Batch 3. Loss (mse): 0.249 <= 0.249 <= 0.249
[AGT] [INFO    ] [12:07:23] Starting epoch 4
[AGT] [INFO    ] [12:07:37] Batch 4. Loss (mse): 0.053 <= 0.053 <= 0.053
[AGT] [INFO    ] [12:09:50] Starting epoch 5
[AGT] [INFO    ] [12:10:04] Batch 5. Loss (mse): 0.076 <= 0.076 <= 0.076
[AGT] [INFO    ] [12:12:14] Starting epoch 6
[AGT] [INFO    ] [12:12:28] Batch 6. Loss (mse): 0.080 <= 0.080 <= 0.081
[AGT] [INFO    ] [12:14:38] Starting epoch 7
[AGT] [INFO    ] [12:14:52] Batch 7. Loss (mse): 0.061 <= 0.062 <= 0.062
[AGT] [INFO    ] [12:17:03] Starting epoch 8
[AGT] [INFO    ] [12:17:1

In [13]:
import torch.nn.functional as F
from abstract_gradient_training.bounded_models import BoundedModel
def noisy_test_mse(
    model: torch.nn.Sequential | BoundedModel,
    batch: torch.Tensor,
    labels: torch.Tensor,
    *,
    noise_level: float | torch.Tensor = 0.0,
    noise_type: str = "laplace",
) -> float:
    """
    Given a pytorch (or bounded) model, calculate the prediction accuracy on a batch of the test set when adding the
    specified noise to the predictions.
    NOTE: For now, this function only supports binary classification via the noise + threshold dp mechanism. This
          should be extended to support multi-class problems via the noisy-argmax mechanism in the future.

    Args:
        model (torch.nn.Sequential | BoundedModel): The model to evaluate.
        batch (torch.Tensor): Input batch of data (shape [batchsize, ...]).
        labels (torch.Tensor): Targets for the input batch (shape [batchsize, ]).
        noise_level (float | torch.Tensor, optional): Noise level for privacy-preserving predictions using the laplace
            mechanism. Can either be a float or a torch.Tensor of shape (batchsize, ).
        noise_type (str, optional): Type of noise to add to the predictions, one of ["laplace", "cauchy"].

    Returns:
        float: The noisy accuracy of the model on the test set.
    """
    # get the test batch and send it to the correct device
    if isinstance(model, BoundedModel):
        device = torch.device(model.device) if model.device != -1 else torch.device("cpu")
    else:
        device = torch.device(next(model.parameters()).device)
    batch = batch.to(device)

    # validate the labels
    if labels.dim() > 1:
        labels = labels.squeeze()
    labels = labels.to(device).type(torch.int64)
    assert labels.dim() == 1, "Labels must be of shape (batchsize, )"

    # validate the noise parameters and set up the distribution
    assert noise_type in ["laplace", "cauchy"], f"Noise type must be one of ['laplace', 'cauchy'], got {noise_type}"
    noise_level += 1e-7  # can't set distributions scale to zero
    noise_level = torch.tensor(noise_level) if isinstance(noise_level, float) else noise_level
    noise_level = noise_level.to(device).type(batch.dtype)  # type: ignore
    noise_level = noise_level.expand(labels.size())
    if noise_type == "laplace":
        noise_distribution = torch.distributions.Laplace(0, noise_level)
    else:
        noise_distribution = torch.distributions.Cauchy(0, noise_level)

    # # nominal, lower and upper bounds for the forward pass
    # logit_n = model.forward(batch).squeeze()

    # # transform 2-logit models to a single output
    # if logit_n.shape[-1] == 2:
    #     logit_n = logit_n[:, 1] - logit_n[:, 0]
    # if logit_n.dim() > 1:
    #     raise NotImplementedError("Noisy accuracy is not supported for multi-class classification.")

    # nominal, lower and upper bounds for the forward pass
    y_n = model.forward(batch).squeeze()

    # transform 2-logit models to a single output
    if y_n.shape[-1] == 2:
        y_n = y_n[:, 1] - y_n[:, 0]
    if y_n.dim() > 1:
        raise NotImplementedError("Noisy accuracy is not supported for multi-class classification.")

    # # apply noise + threshold dp mechanisim
    # y_n = (logit_n > 0).to(torch.float32).squeeze()
    # noise = noise_distribution.sample().to(y_n.device).squeeze()
    # assert noise.shape == y_n.shape
    # y_n = (y_n + noise) > 0.5
    # accuracy = (y_n == labels).float().mean().item()

    # apply noise + threshold dp mechanisim
    noise = noise_distribution.sample().to(y_n.device).squeeze()
    assert noise.shape == y_n.shape
    y_n = y_n + noise
    accuracy = F.mse_loss(y_n, labels.squeeze()).item()
    return accuracy

In [14]:
for k in k_private_values:
    privacy_bounded_models[k].save_params(f"models/18epochs/uci_k{k}.model")

RuntimeError: Parent directory models/18epochs does not exist.

In [ ]:
import importlib
import privacy_utils_regression
importlib.reload(privacy_utils_regression)

epsilon = 1.0
# make privacy-safe predictions using the smooth sensitivity bounds from AGT
noise_level = privacy_utils_regression.get_calibrated_noise_level(
    test_data.tensors[0], privacy_bounded_models, min_bound=0, max_bound=10000, epsilon=epsilon, noise_type="cauchy" 
)
print(noise_level)
accuracy = noisy_test_mse(
    bounded_model, *test_data.tensors, noise_level=noise_level, noise_type="cauchy"
)
print(accuracy / len(test_data))
print(f"Accuracy using AGT smooth sensitivity bounds: {accuracy:.2f}")

ave = 0
num = 10000
for i in range(num):
    ave += noisy_test_mse(
        bounded_model, *test_data.tensors, noise_level=noise_level, noise_type="cauchy"
    )
print(f"Average MSE is {ave / (num * len(test_data))}")

tensor([0.2549, 0.2549, 0.2549,  ..., 0.2549, 0.2549, 0.2549], device='cuda:0')
1.588772794591271
Accuracy using AGT smooth sensitivity bounds: 325584.03
Average MSE is 190.5872059983658


: 